In [1]:
%load_ext autoreload
%autoreload 2

from dataset import CustomImageDataset
import os
import matplotlib.pyplot as plt
import torch

from torchvision.transforms.functional import to_pil_image


In [2]:
from torch.utils.data import DataLoader

training_data = CustomImageDataset(dataset_path="../Datasets/face-emotion", mode="train", transform="grayscale")
test_data = CustomImageDataset(dataset_path="../Datasets/face-emotion", mode="test", transform="grayscale")

train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=4, shuffle=True)

train_features, train_labels = next(iter(training_data))


train_features.shape

c:\Users\pauli\OneDrive - University of Surrey\work\emotion-recognition\dataset.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(


torch.Size([1, 112, 112])

In [3]:
from torch import nn
from torchvision import datasets, transforms

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.flatten = nn.Flatten()
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(112*112*3, 4096),
#             nn.ReLU(),
#             nn.Linear(4096, 2048),
#             nn.ReLU(),
#             nn.Linear(2048, 2),
#         )

#     def forward(self, x):
#         x = self.flatten(x)
#         logits = self.linear_relu_stack(x)
#         return logits
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import xavier_uniform




class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        xavier_uniform(self.conv1.weight)
        self.batch_norm1 = nn.BatchNorm2d(num_features=32)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(num_features=64)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.4)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)

        # Max pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Fully connected layers
        self.linear_shape = 64 * 56 * 56
        self.fc1 = nn.Linear(self.linear_shape, 1024)
        self.fc2 = nn.Linear(1024, 2)


    def forward(self, x):
        # Apply first convolutional layer
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        # Apply second convolutional layer
        x = self.conv2(x)
        # print("current shape: ", x.shape)
        x = self.batch_norm2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        # Apply third convolutional layer
        x = self.conv3(x)

        # Apply max pooling
        x = self.pool(x)

        # print(f"before {x.shape}")
        # Flatten the output for the fully connected layer
        x = x.view(-1, self.linear_shape)
        # print(f"after {x.shape}")
        # Apply first fully connected layer
        x = self.fc1(x)

        # Apply second fully connected layer
        x = self.fc2(x)

        return x


# def forward(self, x):
#     # Apply convolutions and max pooling
#     x = self.pool(F.relu(self.conv1(x)))
#     x = self.pool(F.relu(self.conv2(x)))
#     x = self.pool(F.relu(self.conv3(x)))

#     # Flatten the output for the fully connected layers
#     x = x.view(-1, 128 * 14 * 14)

#     # Fully connected layers with ReLU and output layer
#     x = F.relu(self.fc1(x))
#     x = self.fc2(x)
#     return x

model = NeuralNetwork().to(device)

C:\Users\pauli\AppData\Local\Temp\ipykernel_22284\2122651269.py:29: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  xavier_uniform(self.conv1.weight)


In [6]:
# import timm

# model = timm.create_model('vgg16.tv_in1k', pretrained=True, num_classes=2).to(device)
# model = model.eval()

In [7]:
X = torch.tensor(train_features, dtype=torch.float32, device=device).unsqueeze(0)

logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

C:\Users\pauli\AppData\Local\Temp\ipykernel_22284\644040024.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(train_features, dtype=torch.float32, device=device).unsqueeze(0)


Predicted class: tensor([0], device='cuda:0')


In [8]:
learning_rate = 1e-3
batch_size = 8

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.574331  [    8/  294]
Test Error: 
 Accuracy: 46.7%, Avg loss: 14.424344 

Epoch 2
-------------------------------
loss: 16.359831  [    8/  294]
Test Error: 
 Accuracy: 66.7%, Avg loss: 0.820371 

Epoch 3
-------------------------------
loss: 0.555922  [    8/  294]
Test Error: 
 Accuracy: 46.7%, Avg loss: 0.787643 

Epoch 4
-------------------------------
loss: 0.528066  [    8/  294]
Test Error: 
 Accuracy: 70.0%, Avg loss: 0.652065 

Epoch 5
-------------------------------
loss: 0.267093  [    8/  294]
Test Error: 
 Accuracy: 80.0%, Avg loss: 0.492983 

Epoch 6
-------------------------------
loss: 0.273959  [    8/  294]
Test Error: 
 Accuracy: 80.0%, Avg loss: 0.427597 

Epoch 7
-------------------------------
loss: 0.165858  [    8/  294]
Test Error: 
 Accuracy: 83.3%, Avg loss: 0.973357 

Epoch 8
-------------------------------
loss: 0.067252  [    8/  294]
Test Error: 
 Accuracy: 83.3%, Avg loss: 0.699951 

Epoch 9
--------------

In [14]:
torch.save(model.state_dict(), "./models/paulinho.pt")